In [25]:
from utils import AssessData
from datasets import load_dataset, load_metric
#from sklearn.datasets import fetch_20newsgroups #encountered issues SSLCertificationError with this 
from datasets import load_dataset
import os
import pandas as pd
import re
from bs4 import BeautifulSoup
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding

from spacy import tokenizer
from spacy.lang.en import English
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer,AdamW,get_scheduler


In [3]:
f1_score = load_metric("f1")

def compute_metrics(eval_pred):
    predictions, label = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return f1_score.compute(predictions = predictions, references = label)

In [40]:
hyperpartisan_dataset = load_dataset("hyperpartisan_news_detection", "bypublisher", "/data") #Not stripped of headers and footers
hyperpartisan_dataset.keys()

Using custom data configuration bypublisher-data_dir=%2Fdata
Reusing dataset hyperpartisan_news_detection (/Users/max/.cache/huggingface/datasets/hyperpartisan_news_detection/bypublisher-data_dir=%2Fdata/1.0.0/7f4215b0474950ddf516e806400ab81d098b3da3b3a919a13cd1a4cf2c677012)
100%|██████████| 2/2 [00:00<00:00, 25.08it/s]


dict_keys(['train', 'validation'])

In [7]:
import json

In [8]:
distribution_name = "hyperpartisan_distribution"

with open(distribution_name, 'r') as ins:
    distribution= json.load(ins)

seed = distribution['seed']

train_index = distribution['train']
eval_index = distribution['eval']
test_index = distribution['test']

In [15]:
train_index_for_longformers = train_index['Short'] + train_index["BERT"] + train_index['Long']
eval_index_for_longformers = eval_index['Short'] + eval_index["BERT"] + eval_index['Long']
test_index_for_longformers = test_index['Short'] + test_index["BERT"] + test_index['Long']

In [41]:
#change to bypublisher because byarticle has no test set.
hyperpartisan_train = hyperpartisan_dataset['train']
hyperpartisan_valid = hyperpartisan_dataset['validation']

hyperpartisan_train = hyperpartisan_train.shuffle(seed = seed)
hyperpartisan_valid = hyperpartisan_valid.shuffle(seed = seed)

Loading cached shuffled indices for dataset at /Users/max/.cache/huggingface/datasets/hyperpartisan_news_detection/bypublisher-data_dir=%2Fdata/1.0.0/7f4215b0474950ddf516e806400ab81d098b3da3b3a919a13cd1a4cf2c677012/cache-fd41281fa7a8783e.arrow
Loading cached shuffled indices for dataset at /Users/max/.cache/huggingface/datasets/hyperpartisan_news_detection/bypublisher-data_dir=%2Fdata/1.0.0/7f4215b0474950ddf516e806400ab81d098b3da3b3a919a13cd1a4cf2c677012/cache-ef73cd395e00d405.arrow


In [42]:
#Split validation data into test and train_set
eval_test_split = hyperpartisan_valid.train_test_split(test_size = 0.5)

hyperpartisan_eval = eval_test_split['train']
hyperpartisan_test = eval_test_split['test']

In [43]:
int_labels_train = [1 if x == True else 0 for x in hyperpartisan_train['hyperpartisan']]
int_labels_eval = [1 if x == True else 0 for x in hyperpartisan_eval['hyperpartisan']]
int_labels_test = [1 if x == True else 0 for x in hyperpartisan_test['hyperpartisan']]

In [44]:
hyperpartisan_train = hyperpartisan_train.add_column("labels", int_labels_train)
hyperpartisan_eval= hyperpartisan_eval.add_column("labels", int_labels_eval)
hyperpartisan_test = hyperpartisan_test.add_column("labels", int_labels_test)

Loading cached processed dataset at /Users/max/.cache/huggingface/datasets/hyperpartisan_news_detection/bypublisher-data_dir=%2Fdata/1.0.0/7f4215b0474950ddf516e806400ab81d098b3da3b3a919a13cd1a4cf2c677012/cache-6e59f494bef5bf5e.arrow
Flattening the indices: 100%|██████████| 300/300 [00:34<00:00,  8.73ba/s]


In [45]:
hyperpartisan_train = hyperpartisan_train.remove_columns(["title","hyperpartisan","url","published_at","bias"])
hyperpartisan_eval = hyperpartisan_eval.remove_columns(["title","hyperpartisan","url","published_at","bias"])
hyperpartisan_test = hyperpartisan_test.remove_columns(["title","hyperpartisan","url","published_at","bias"])

In [46]:
hyperpartisan_train = hyperpartisan_train.select(train_index_for_longformers)
hyperpartisan_eval = hyperpartisan_eval.select(eval_index_for_longformers)
hyperpartisan_test = hyperpartisan_test.select(test_index_for_longformers)

In [ ]:
#extract long documents from test and eval
#evaluate on test and eval
#Use test_json to have index of long_files and short files and normal files as well as labels

In [21]:
#FINETUNING TO A NUMBER OF Labels
from transformers import LongformerTokenizer, LongformerForSequenceClassification,LongformerConfig

longformer_tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')

longformer_model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096',
                                                  gradient_checkpointing=False,
                                                  attention_window = 512,
                                                  num_labels = 2,
                                                  return_dict=True)


Downloading: 100%|██████████| 570M/570M [00:38<00:00, 15.5MB/s] 
Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly in

In [ ]:
with torch.no_grad():
  input = longformer_tokenizer(hyperpartisan_train['text'], return_tensors= "pt")
  out = longformer_model(**input).logits

  labels = out.argmax().item()

In [ ]:
labels, hyperpartisan_train['hyperpartisan']

FINETUNING

In [22]:
import optuna

In [23]:
def hp_space(trial):
  return {"per_device_train_batch_size":trial.suggest_int("per_device_train_batch_size", 8,16,8),
          "num_train_epochs":trial.suggest_int("num_training_epochs",1,5),
          "learning_rate": trial.suggest_float("learning_rate",1e-5,2e-5),
  }

In [47]:

def longformer_init():
  return (LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096',
                                                  gradient_checkpointing=False,
                                                  attention_window = 512,
                                                  num_labels = 2,
                                                  return_dict=True))
                                                                     
data_collator = DataCollatorWithPadding(tokenizer=longformer_tokenizer) #to pad sequences

#default training arguments 
training_args = TrainingArguments(output_dir="./results", per_device_train_batch_size=16, 
                                  evaluation_strategy= "epoch", per_device_eval_batch_size=16, num_train_epochs=5)

#fine-tuning model
longformer_finetuned = Trainer(model_init = longformer_init, 
              model = longformer_model,
              args = training_args,
              train_dataset= hyperpartisan_train,
              eval_dataset= hyperpartisan_eval,
              tokenizer= longformer_tokenizer,
              data_collator=data_collator,
              compute_metrics= compute_metrics)

#searching for best hyperparameters
best_run = longformer_finetuned.hyperparameter_search(direction = "maximize", hp_space = hp_space)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/Users/max/Desktop/Dissertation/implementations/Disso-COLD/proj_env/lib/python3.8/site-packages/transformers/trainer.py:330: FutureWarning: `Trainer` requires either a `model` or `model_init` argument, but not both. `model_init` will overwrite your model when calling the `train` method. This will become a fatal error in the next release.
  warnings.warn(
  0%|          | 0/73606 [18:45<?, ?it/s]
[I 2022-07-28 13:26:26,879] A new study created in memory with name: no-name-e40b98a4-3e92-4db5-9728-e3a5a60683b5
Trial:
loading configuration file https://huggingface.co/allenai/longformer-base-4096/resolve/main/config.json from cache at /Users/max/.cache/huggingface/transformers/0690955d8f70934f95adf0f

ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['labels']

In [ ]:
#plot 
fig, ax = optuna.visualization.plot_slice(study, params=["x", "y"])

In [ ]:
#update hyperparameters 
for k,v in best_run.hyperparameters.items():
  setattr(training_args, k, v)

best_trainer = Trainer(model = longformer_model,
                       args = training_args,
                       train_dataset= hyperpartisan_train,
                      eval_dataset= hyperpartisan_eval,
                      tokenizer= longformer_tokenizer,
                      data_collator=data_collator,
                      compute_metrics= compute_metrics)

best_trainer.train()